In [1]:
import pandas as pd
from sklearn.linear_model import Ridge

In [20]:
item_based_df = pd.read_csv('outputs/out_2_1.csv')
item_based_df.head()

,user_id,business_id,prediction
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,4.499354
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,4.713610
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.365733
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,4.860414
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,2.929358


In [21]:
model_based_df = pd.read_csv(
    'outputs/out_2_2.csv')
model_based_df.head()

,user_id,business_id,prediction
0,DoRCeCcJbrsM2BiAKj3trA,7xLWtcBooa2op1-wIOtWdQ,4.181924
1,bLbSNkLggFnqwNNzzq-Ijw,7xLWtcBooa2op1-wIOtWdQ,3.670336
2,L6nrDwHJDyV0KThjON-Kog,7xLWtcBooa2op1-wIOtWdQ,4.427603
3,tH0uKD-vNwMoEc3Xk3Cbdg,7xLWtcBooa2op1-wIOtWdQ,4.348206
4,OLn8EvPsu4hNug8V5PF2jA,7xLWtcBooa2op1-wIOtWdQ,3.620203


In [22]:
data = pd.read_csv(
    'resource/asnlib/publicdata/yelp_val.csv')
data.head()

,user_id,business_id,stars
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,3.0


In [23]:
item_based_df.count()
model_based_df.count()
# als_based_df.count()

user_id        142044
business_id    142044
prediction     142044
dtype: int64

In [24]:
combined_rating = pd.merge(left=item_based_df, right=model_based_df, on=['user_id', 'business_id']).rename(
    columns={'prediction_x': 'item_based_rating', 'prediction_y': 'model_based_rating'})
combined_rating = pd.merge(left=combined_rating, right=data, on=['user_id', 'business_id'])
# combined_rating = pd.merge(left=combined_rating, right=als_based_df, on=['user_id', 'business_id']).rename(
#     columns={'prediction': 'als_based_rating'})
combined_rating = combined_rating[['user_id', 'business_id', 'item_based_rating', 'model_based_rating', 'stars']]
combined_rating.head()

,user_id,business_id,item_based_rating,model_based_rating,stars
0,wf1GqnKQuvH-V3QN80UOOQ,fThrN4tfupIGetkrz18JOg,4.499354,3.805042,5.0
1,39FT2Ui8KUXwmUt6hnwy-g,uW6UHfONAmm8QttPkbMewQ,4.713610,4.796097,5.0
2,7weuSPSSqYLUFga6IYP4pg,IhNASEZ3XnBHmuuVnWdIwA,4.365733,4.797669,4.0
3,CqaIzLiWaa-lMFYBAsYQxw,G859H6xfAmVLxbzQgipuoA,4.860414,4.738786,5.0
4,yy7shAsNWRbGg-8Y67Dzag,rS39YnrhoXmPqHLzCBjeqw,2.929358,2.975218,3.0


In [25]:
combined_rating[['item_based_rating']].describe()

,item_based_rating
count,142044.000000
mean,3.748541
std,0.528264
min,0.279157
25%,3.428250
50%,3.785075
75%,4.121603
max,5.000000


In [26]:
combined_rating[['model_based_rating']].describe()

,model_based_rating
count,142044.000000
mean,3.751624
std,0.536716
min,1.000000
25%,3.425839
50%,3.807369
75%,4.139801
max,5.000000


In [27]:
# Choose alpha through CV
alpha_range = [0.01, 0.1, 1, 10, 100]
best_alpha = None
best_mse = float('inf')
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error

kf = KFold(n_splits=5)
for alpha in alpha_range:
    mse_list = []
    for train_index, test_index in kf.split(combined_rating):
        train, test = combined_rating.iloc[train_index], combined_rating.iloc[test_index]
        lr = Ridge(alpha=alpha, fit_intercept=False)
        lr.fit(train[['item_based_rating', 'model_based_rating']], train[['stars']])
        y_pred = lr.predict(test[['item_based_rating', 'model_based_rating']])
        mse = root_mean_squared_error(test[['stars']], y_pred)
        mse_list.append(mse)
    avg_mse = sum(mse_list) / len(mse_list)
    if avg_mse < best_mse:
        best_mse = avg_mse
        best_alpha = alpha

print(f'Best alpha: {best_alpha}, Best MSE: {best_mse}')

Best alpha: 10, Best MSE: 0.9827894961996387


In [28]:
#Fit a linear regression model without intercept
lr = Ridge(fit_intercept=True, alpha=10)
lr.fit(combined_rating[['item_based_rating', 'model_based_rating']], combined_rating[['stars']])
lr

Ridge(alpha=10)

In [29]:
# Output the linear regression model parameters
print(f'Coefficients: {lr.coef_}')
print(f"Interscept: {lr.intercept_}")
print(
    f"Equation: stars = {lr.coef_[0][0]} * item_based_rating + {lr.coef_[0][1]} * model_based_rating + {lr.intercept_[0]}")

Coefficients: [[0.09445333 0.94178587]]
Interscept: [-0.13678897]
Equation: stars = 0.09445333302830297 * item_based_rating + 0.9417858699086205 * model_based_rating + -0.13678896605186797


In [30]:
y_pred = lr.predict(combined_rating[['item_based_rating', 'model_based_rating']])
combined_rating['predicted_stars'] = y_pred

mse = root_mean_squared_error(combined_rating['stars'], combined_rating['predicted_stars'])

print(f'Root Mean Squared Error: {mse}')

Root Mean Squared Error: 0.982608482475758


In [31]:
# Fit a XGBoost model
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(combined_rating[['item_based_rating', 'model_based_rating']], combined_rating['stars'])

/Users/navneet/anaconda3/envs/dsci_553/lib/python3.9/site-packages/xgboost/core.py:195: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


XGBRegressor()

In [32]:
y_pred = xgb.predict(combined_rating[['item_based_rating', 'model_based_rating']])
combined_rating['predicted_stars_xgb'] = y_pred

mse = root_mean_squared_error(combined_rating['stars'], combined_rating['predicted_stars_xgb'])

print(f'Root Mean Squared Error: {mse}')

Root Mean Squared Error: 0.9801100345047365


/Users/navneet/anaconda3/envs/dsci_553/lib/python3.9/site-packages/xgboost/core.py:195: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


In [33]:
"""
There are in general two ways that you can control overfitting in XGBoost:

The first way is to directly control model complexity.

This includes max_depth, min_child_weight and gamma.
"""

# Cross validation
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [1, 3, 5, 7, 9],
    'min_child_weight': [1, 3, 5, 7]
}

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(combined_rating[['item_based_rating', 'model_based_rating']],
                combined_rating['stars'])

Fitting 5 folds for each of 20 candidates, totalling 100 fits


python(88791) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88792) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88793) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88794) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88795) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88796) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88797) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88798) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/navneet/anaconda3/envs/dsci_553/lib/python3.9/site-packages/xgboost/core.py:195: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(

GridSearchCV(cv=5, estimator=XGBRegressor(), n_jobs=-1,
             param_grid={'max_depth': [1, 3, 5, 7, 9],
                         'min_child_weight': [1, 3, 5, 7]},
             verbose=2)

In [35]:
grid_search.best_params_

{'max_depth': 3, 'min_child_weight': 1}

In [36]:
xgb = XGBRegressor(max_depth=3, min_child_weight=1)
xgb.fit(combined_rating[['item_based_rating', 'model_based_rating']], combined_rating['stars'])

/Users/navneet/anaconda3/envs/dsci_553/lib/python3.9/site-packages/xgboost/core.py:195: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


XGBRegressor()

In [37]:
y_pred = xgb.predict(combined_rating[['item_based_rating', 'model_based_rating']])
combined_rating['predicted_stars_xgb'] = y_pred

mse = root_mean_squared_error(combined_rating['stars'], combined_rating['predicted_stars_xgb'])

print(f'Root Mean Squared Error: {mse}')

Root Mean Squared Error: 0.9801100345047365


/Users/navneet/anaconda3/envs/dsci_553/lib/python3.9/site-packages/xgboost/core.py:195: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()
